## タイタニック号の生存者予測
データ分析コンペティションプラットフォームKaggleのチュートリアルとしても使われるタイタニック号の生存者予測を行う．評価は予測された生存確率と実際に生存したかどうかの値を使って計算される AUC（Area under the curve） Score を用いて評価をする．予測する値は生死の 0, 1 ではなく生き残る確率値0<=y<=1とする．また、モデルにはKernel Ridge Regressionを使用する。

### データセット
トレインデータとその特徴量の種類を以下に示す．

**特徴量の種類**
- PassengerId : 乗客のID
- Survived : 生死(0:死亡,1:生存)
- Pclass : 乗客の社会階級(1:Upper, 2:Middle, 3:Lower)
- Name : 名前
- Sex : 性別
- Age : 年齢
- SibSp : 一緒に乗っている兄弟、配偶者の数
- Parch : 一緒に乗っている親、子供の数
- Ticket : チケットの番号
- Fare : 乗船料
- Cabin : 客室番号
- Embarked : 乗船した港

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

df = pd.read_csv('./dataset/train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### 前処理
欠損値を埋め、不要な特徴を削除し、使用する特徴量を表示する

In [2]:
df["Age"] = df["Age"].fillna(df["Age"].median())  # 欠損値を埋める
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])  # 欠損値を埋める
features= df.columns.values[2:]
features=np.delete(features,1)  # Nameを削除
df_train,df_test=train_test_split(df,train_size=0.8,random_state=1)
print('使用する特徴量',features)

使用する特徴量 ['Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin' 'Embarked']


### Wrapper Methodを行う関数の定義
特徴量選択手法の1つである Wrapper Methodを用いて、最適な特徴量の組合せを探索する。
評価には10分割のクロスバリデーションを用いる。

In [3]:
def wrapper_method(features,df):
    selected_features = []
    auc_iteration = []
    for i in range(len(features)):
        print('iteration:',i)
        auc_list=[]
        for column in features:
            variable_dummy=pd.DataFrame()
            variable_numeral=pd.DataFrame()
            if df[column].dtype==object:
                variable_dummy = pd.get_dummies(df[[column] + [feature for feature in selected_features if df[feature].dtype == object]], dummy_na=True)
                feature_numeral=[feature for feature in selected_features if df[feature].dtype != object]
                if len(feature_numeral)!=0:
                    variable_numeral = pd.get_dummies(df[feature_numeral], dummy_na=True)
            else:
                variable_numeral = df[[column] + [feature for feature in selected_features if df[feature].dtype != object]]
                feature_symbolic=[feature for feature in selected_features if df[feature].dtype == object]
                if len(feature_symbolic)!=0:
                    variable_dummy = pd.get_dummies(df[feature_symbolic], dummy_na=True)
                if i==0:
                    variable_numeral=variable_numeral.values.reshape(-1, 1)
            if not variable_dummy.empty and not variable_numeral.empty:
                x=pd.concat([variable_numeral, variable_dummy],axis=1)
            elif variable_dummy.empty:
                x=variable_numeral
            else:
                x=variable_dummy
            y=df['Survived']
            model = KernelRidge(kernel='rbf')
            scores = cross_val_score(model, x, y, scoring='roc_auc', cv=10)
            auc_list.append(scores.mean())

        max_feature_index=auc_list.index(max(auc_list))
        selected_features.append(features[max_feature_index])
        print('max AUC:',max(auc_list),'    feature:',selected_features)
        if i != 0:
            if max(auc_iteration) >= max(auc_list):
                print('**** stop iteration ****')
                return selected_features[:-1]
        auc_iteration.append(max(auc_list))
        features=np.delete(features,max_feature_index)
    return selected_features

### Wrapper Methodの実行
Wrapper Methodを実行し、特徴量選択の結果を表示する。
iteration毎に、予測への貢献が最も高い特徴量が追加され、評価指標が悪化した時点で、iterationが終わる。

In [4]:
selected_features=wrapper_method(features,df_train)
print('Wrapper Methodによって選択された特徴量',selected_features)

iteration: 0
max AUC: 0.7750050910102636     feature: ['Sex']
iteration: 1
max AUC: 0.8422942758619228     feature: ['Sex', 'Pclass']
iteration: 2
max AUC: 0.8605279928868459     feature: ['Sex', 'Pclass', 'Ticket']
iteration: 3
max AUC: 0.8605285659911607     feature: ['Sex', 'Pclass', 'Ticket', 'Embarked']
iteration: 4
max AUC: 0.86032858508696     feature: ['Sex', 'Pclass', 'Ticket', 'Embarked', 'Cabin']
**** stop iteration ****
Wrapper Methodによって選択された特徴量 ['Sex', 'Pclass', 'Ticket', 'Embarked']


### 学習
Wrapper Methodで選択された特徴量をもとにモデルの学習を行う。

In [5]:
def train_model(df,selected_features):
    variable_dummy = pd.DataFrame()
    variable_numeral = pd.DataFrame()
    feature_symbolic = [feature for feature in selected_features if df[feature].dtype == object]
    if len(feature_symbolic) != 0:
        variable_dummy = pd.get_dummies(df[feature_symbolic], dummy_na=True)
    feature_numeral = [feature for feature in selected_features if df[feature].dtype != object]
    if len(feature_numeral) != 0:
        variable_numeral = pd.get_dummies(df[feature_numeral], dummy_na=True)
    if not variable_dummy.empty and not variable_numeral.empty:
        x = pd.concat([variable_numeral, variable_dummy], axis=1)
    elif variable_dummy.empty:
        x = variable_numeral
    else:
        x = variable_dummy
    y = df['Survived']
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)
    model = KernelRidge(kernel='rbf')
    model.fit(x_train, y_train)
    return model,x_test,y_test

model,x_test,y_test=train_model(pd.concat([df_train, df_test]),selected_features)

### 評価
テストデータでモデルの評価を行う。

In [6]:
def predict_test(x_test,y_test,selected_features,model):
    pred=model.predict(x_test)
    print('AUC:',roc_auc_score(y_test, pred))
    
predict_test(x_test,y_test,selected_features,model)

AUC: 0.8987811340752516
